#![Spark Logo](http://spark-mooc.github.io/web-assets/images/ta_Spark-logo-small.png) + ![Python Logo](http://spark-mooc.github.io/web-assets/images/python-logo-master-v3-TM-flattened_small.png)
#Practica sobre como generar un flujo de ejecucion en un problema de Machine Learning

Esta practica simula un ejercicio completo de ETL (Extract-Transform-Load) junto a un analisis exploratorio de un dataset real, para posteriormente aplicar differentes algoritmos de aprendizaje automatico que resuelvan un problema de regresion.

** This notebook covers: **
* *Parte 1: Conocimiento del dominio*
* *Parte 2: Extraccion, transformacion y carga [ETL] del dataset* (1 punto sobre 10)
* *Parte 3: Explorar los datos* (1 puntos sobre 10)
* *Parte 4: Visualizar los datos* (1 puntos sobre 10)
* *Parte 5: Preparar los datos* (1 puntos sobre 10)
* *Parte 6: Modelar los datos* (2 puntos sobre 10)
* *Parte 7: Ajustar y evaluar* (4 puntos sobre 10)

*Nuestro objetivo sera predecir de la forma mas exacta posible la energia generada por un conjunto de de plantas electricas usando los datos generados por un conjunto de sensores.*


## Parte 1: Conocimiento del dominio

** Background **


La generacion de energia es un proceso complejo, comprenderlo para poder predecir la potencia de salida es un elemento vital en la gestion de una planta energetica y su conexion a la red. Los operadores de una red electrica regional crean predicciones de la demanda de energia en base a la informacion historica y los factores ambientales (por ejemplo, la temperatura). Luego comparan las predicciones con los recursos disponibles (por ejemplo, plantas, carbon, gas natural, nuclear, solar, eolica, hidraulica, etc). Las tecnologias de generacion de energia, como la solar o la eolica, dependen en gran medida de las condiciones ambientales, pero todas las centrales electricas son objeto de mantenimientos tanto planificados y como puntuales debidos a un problema.

En esta practica usaremos un ejemplo del mundo real sobre la demanda prevista (en dos escalas de tiempo), la demanda real, y los recursos disponibles de la red electrica de California: http://www.caiso.com/Pages/TodaysOutlook.aspx

![](http://content.caiso.com/outlook/SP/ems_small.gif)

El reto para un operador de red de energia es como manejar un deficit de recursos disponibles frente a la demanda real. Hay tres posibles soluciones a un deficit de energia: construir mas plantas de energia base (este proceso puede costar muchos anos de planificacion y construccion), comprar e importar de otras redes electricas regionales energia sobrabte (esta opcion puede ser muy cara y esta limitado por las interconexiones entre las redes de transmision de energia y el exceso de potencia disponible de otras redes), o activar pequenas [plantas de pico](https://en.wikipedia.org/wiki/Peaking_power_plant). Debido a que los operadores de red necesitan responder con rapidez a un deficit de energia para evitar un corte del suministro, estos basan sus decisiones en una combinacion de las dos ultimas opciones. En esta practica, nos centraremos en la ultima eleccion.

** La logica de negocio **

Debido a que la demanda de energia solo supera a la oferta ocasionalmente, la potencia suministrada por una planta de energia pico tiene un precio mucho mas alto por kilovatio hora que la energia generada por las centrales electricas base de una red electrica. Una planta pico puede operar muchas horas al dia, o solo unas pocas horas al ano, dependiendo de la condicion de la red electrica de la region. Debido al alto coste de la construccion de una planta de energia eficiente, si una planta pico solo va a funcionar por un tiempo corto o muy variable, no tiene sentido economico para que sea tan eficiente como una planta de energia base. Ademas, el equipo y los combustibles utilizados en las plantas base a menudo no son adecuados para uso en plantas de pico.

La salida de potencia de una central electrica pico varia dependiendo de las condiciones ambientales, por lo que el problema de negocio a resolver se podria describir como _predecir la salida de potencia de una central electrica pico en funcion de la condiciones ambientales_  - ya que esto permitiria al operador de la red hacer compensaciones economicas sobre el numero de plantas pico que ha de conectar en cada momento (o si por el contrario le interesa comprar energia mas cara de otra red).

Una vez descrita esta logica de negocio, primero debemos proceder a realizar un analisis exploratorio previo y trasladar el problema de negocio (predecir la potencia de salida en funcion de las condiciones medio ambientales) en un tarea de aprendizaje automatico (ML). Por ejemplo, una tarea de ML que podriamos aplicar a este problema es la regression, ya que tenemos un variable objetivo (dependiente) que es numerica. Para esto usaremos [Apache Spark ML Pipeline](https://spark.apache.org/docs/1.6.2/api/python/pyspark.ml.html#pyspark-ml-package) para calcular dicha regresion.

Los datos del mundo real que usaremos en esta practica se componen de 9.568 puntos de datos, cada uno con 4 atributos ambientales recogidos en una Central de Ciclo Combinado de mas de 6 anos (2006-2011), proporcionado por la Universidad de California, Irvine en [UCI Machine Learning Repository Combined Cycle Power Plant Data Set](https://archive.ics.uci.edu/ml/datasets/Combined+Cycle+Power+Plant)). Para mas detalles sobre el conjunto de datos visitar la pagina de la UCI, o las siguientes referencias:

* Pinar Tufekci, [Prediction of full load electrical power output of a base load operated combined cycle power plant using machine learning methods](http://www.journals.elsevier.com/international-journal-of-electrical-power-and-energy-systems/), International Journal of Electrical Power & Energy Systems, Volume 60, September 2014, Pages 126-140, ISSN 0142-0615.
* Heysem Kaya, Pinar Tufekci and Fikret S. Gurgen: [Local and Global Learning Methods for Predicting Power of a Combined Gas & Steam Turbine](http://www.cmpe.boun.edu.tr/~kaya/kaya2012gasturbine.pdf), Proceedings of the International Conference on Emerging Trends in Computer and Electronics Engineering ICETCEE 2012, pp. 13-18 (Mar. 2012, Dubai).

**Ejercicio 1(a)**: Leer la documentacion y referencias de [Spark Machine Learning Pipeline](https://spark.apache.org/docs/1.6.2/ml-guide.html#main-concepts-in-pipelines).

## Part 2: Extraccion, transformacion y carga [ETL] del dataset


Ahora que entendemos lo que estamos tratando de hacer, el primer paso consiste en cargar los datos en un formato que podemos consultar y utilizar facilmente. Esto se conoce como ETL o "extraccion, transformacion y carga". Primero, vamos a cargar nuestro archivo de Amazon S3.

Nota: Como alternativa podemos subir nuestros datos utilizando "Databricks Menu> Tablas> Crear tabla", suponiendo que tengamos los archivos sin procesar en nuestro ordenador local.

Nuestros datos esta disponible en Amazon S3 en la siguiente ruta:

```
dbfs:/databricks-datasets/power-plant/data
```

**Ejercicio 1(b):** Empezaremos por visualizar una muestra de los datos. Para esto usaremos las funciones pre-definidas en los notebooks de Databricks para explorar su sistema de archivos.

Usar `display(dbutils.fs.ls("/databricks-datasets/power-plant/data"))` para listar los ficheros del directorio

In [3]:
#TODO: use display to list all the files of the directory containing the data
<FILL_IN>

Ahora, usaremos el comando `dbutils.fs.head` para ver los primeros 65,536 bytes del primer archivo del directorio.

Usar `print dbutils.fs.head("/databricks-datasets/power-plant/data/Sheet1.tsv")` para ver el contenido del archivo Sheet1.tsv

In [5]:
#TODO: print the first 65,536 bytes of the file Sheet1.tsv
print <FILL_IN>

`dbutils.fs` dispone de su propio help, esta ayuda nos sera de gran utilidad cuando deseemos ver las diferentes funciones disponibles.

In [7]:
dbutils.fs.help()

### Ejercicio 2(a)

Ahora usaremos PySpark para visualizar las 5 primeras lineas de los datos

*Hint*: Primero crea un RDD a partir de los datos usando [`sc.textFile("dbfs:/databricks-datasets/power-plant/data")`](https://spark.apache.org/docs/1.6.2/api/python/pyspark.html#pyspark.SparkContext.textFile).

*Hint*: Luego piensa como usar el RDD creado para mostrar datos, el metodo [`take()`](https://spark.apache.org/docs/1.6.2/api/python/pyspark.html#pyspark.RDD.take) puede ser una buena opcion a considerar.

In [9]:
# TODO: Load the data and print the first five lines.
rawTextRdd = <FILL_IN>

A partir nuestra exploracion inicial de una muestra de los datos, podemos hacer varias observaciones sobre el proceso de ETL:
- Los datos son un conjunto de .tsv (archivos con valores separados Tab) (es decir, cada fila de datos se separa mediante tabuladores)
- Hay una fila de cabecera, que es el nombre de las columnas
- Parece que el tipo de los datos en cada columna es constante (es decir, cada columna es de tipo double)

El esquema de datos que hemos obtenido de UCI es:
- AT = Atmospheric Temperature in C
- V = Exhaust Vacuum Speed
- AP = Atmospheric Pressure
- RH = Relative Humidity
- PE = Power Output.  Esta es la variable dependiente que queremos predecir usando los otras cuatro


Ahora estamos en disposicion de crear un DataFrame a partir de los datos de TSV. Spark no tiene un metodo nativo para realizar esta operacion, sin embargo, podemos utilizar [spark-csv](https://spark-packages.org/package/databricks/spark-csv), un paquete de un tercero de [SparkPackages](https://spark-packages.org/). La documentacion y el codigo fuente para [spark-csv](https://spark-packages.org/package/databricks/spark-csv) pueden encontrarse en [GitHub](https://github.com/databricks/spark-csv). La API de Python puede encontrarse [aqui](https://github.com/databricks/spark-csv#python-api).

(**Nota**: En Spark 2.0, El paquete CSV se encuenta dentro de las llamadas de la API de la clase DataFrame.)

Para usar el paquete Spark CSV [spark-csv](https://spark-packages.org/package/databricks/spark-csv), usaremos el metodo [sqlContext.read.format()](https://spark.apache.org/docs/1.6.2/api/python/pyspark.sql.html#pyspark.sql.DataFrameReader.format) para especificar el formato de la fuente de datos de entrada: `'com.databricks.spark.csv'`

Podemos especificar diferentes opciones de como importar los datos usando el metodo [options()](https://spark.apache.org/docs/1.6.2/api/python/pyspark.sql.html#pyspark.sql.DataFrameReader.options). Encontramos las opciones disponible en la documentacion de GitHub del paquete [aqui](https://github.com/databricks/spark-csv#features).

Usaremos las siguientes opciones:
- `delimiter='\t'` porque nuestros datos se encuentran delimitados por tabulaciones
- `header='true'` porque nuestro dataset tiene una fila que representa la cabezera de los datos
- `inferschema='true'` porque creemos que todos los datos son numeros reales, por lo tanto la libreria puede inferir el tipo de cada columna de forma automatica.

El ultimo componente necesario para crear un DataFrame es determinar la ubicacion de los datos usando el metodo [load()](https://spark.apache.org/docs/1.6.2/api/python/pyspark.sql.html#pyspark.sql.DataFrameReader.load): `"/databricks-datasets/power-plant/data"`

Juntando todo, usaremos la siguiente operacion:

`sqlContext.read.format().options().load()`

### Exercicio 2(b)

**To Do:** Crear un DataFrame a partir de los datos.

In [11]:
# TODO: Replace <FILL_IN> with the appropriate code.
powerPlantDF = sqlContext.read.format(<FILL_IN>).options(<FILL_IN>).load(<FILL_IN>)

In [12]:
# TEST
from databricks_test_helper import *
expected = set([(s, 'double') for s in ('AP', 'AT', 'PE', 'RH', 'V')])
Test.assertEquals(expected, set(powerPlantDF.dtypes), "Incorrect schema for powerPlantDF")

Vamos a comprobar los tipos de las columnas usando el metodo [dtypes](https://spark.apache.org/docs/1.6.2/api/python/pyspark.sql.html#pyspark.sql.DataFrame.dtypes).

In [14]:
print powerPlantDF.dtypes

Tambien podemos examinar los datos usando el metodo `display()`.

In [16]:
display(powerPlantDF)

Ahora en lugar de usar [spark-csv](https://spark-packages.org/package/databricks/spark-csv) para inferir los tipos de las columnas, especificaremos el esquema como [DataType](https://spark.apache.org/docs/1.6.2/api/python/pyspark.sql.html#pyspark.sql.types.DataType), el cual es una lista de [StructField](https://spark.apache.org/docs/1.6.2/api/python/pyspark.sql.html#pyspark.sql.types.StructType).

La lista completa de tipos se encuetra en el modulo [pyspark.sql.types](https://spark.apache.org/docs/1.6.2/api/python/pyspark.sql.html#module-pyspark.sql.types). Para nuestros datos, usaremos [DoubleType()](https://spark.apache.org/docs/1.6.2/api/python/pyspark.sql.html#pyspark.sql.types.DoubleType).

Por ejemplo, para especificar cual es el nombre de la columna usaremos: `StructField(`_name_`,` _type_`, True)`. (El tercer parametro, `True`, significa que permitimos que la columna tenga valores null.)

### Exercicio 2(c)

Crea un esquema a medida para el dataset.

In [18]:
# TO DO: Fill in the custom schema.
from pyspark.sql.types import *

# Custom Schema for Power Plant
customSchema = StructType([ \
    <FILL_IN>, \
    <FILL_IN>, \
    <FILL_IN>, \
    <FILL_IN>, \
    <FILL_IN> \
                          ])

In [19]:
# TEST
Test.assertEquals(set([f.name for f in customSchema.fields]), set(['AT', 'V', 'AP', 'RH', 'PE']), 'Incorrect column names in schema.')
Test.assertEquals(set([f.dataType for f in customSchema.fields]), set([DoubleType(), DoubleType(), DoubleType(), DoubleType(), DoubleType()]), 'Incorrect column types in schema.')

### Exercicio 2(d)

Ahora, usaremos el esquema que acabamos de crear para leer los datos. Para realizar esta operacion, modificaremos el paso anterior `sqlContext.read.format`. Podemos especificar el esquema haciendo:
- Anadir `schema = customSchema` al metodo load (simplemente anadelo usando una coma justo despues del nombre del archivo)
- Eliminado la opcion `inferschema='true'` ya que ahora especificamos el esquema que han de seguir los datos

In [21]:
# TODO: Use the schema you created above to load the data again.
altPowerPlantDF = sqlContext.read.format(<FILL_IN>).options(<FILL_IN>).load(<FILL_IN>)

In [22]:
# TEST
from databricks_test_helper import *
expected = set([(s, 'double') for s in ('AP', 'AT', 'PE', 'RH', 'V')])
Test.assertEquals(expected, set(altPowerPlantDF.dtypes), "Incorrect schema for powerPlantDF")

Es importante darse cuenta que esta vez no se ha ejecutado ningun job de Spark. Esto se debe a que hemos especificado el esquema, por tanto el paquete [spark-csv](https://spark-packages.org/package/databricks/spark-csv) no tiene porque leer los datos para inferir el esquema. Podemos usar el metodo [dtypes](https://spark.apache.org/docs/1.6.2/api/python/pyspark.sql.html#pyspark.sql.DataFrame.dtypes) para examinar el nombre y el tipo de los atributos del dataset. Estos deberian ser identicos a los que hemos inferido anteriormente de los datos.

Cuando ejecutes la siguiente celda, los datos no deberian leerse.

In [24]:
print altPowerPlantDF.dtypes

Ahora podemos examinar los datos utilizando el metodo display(). * Ten en cuenta que esta operacion hara que los datos que se lean y se creara el DataFrame. *

In [26]:
display(altPowerPlantDF)

## Parte 3: Explorar tus Datos

Ahora que ya hemos cargado los datos, el siguiente paso es explorarlos y realizar algunos analisis y visualizaciones basicas.

Este es un paso que siempre se debe realizar **antes de** intentar ajustar un modelo a los datos, ya que este paso muchas veces nos permitira conocer una gran informacion sobre los datos.

En primer lugar vamos a registrar nuestro DataFrame como una tabla de SQL llamado `power_plant`. Debido a que es posible que repitas esta practica varias veces, vamos a tomar la precaucion de eliminar cualquier tabla existente en primer lugar.

Podemos eliminar cualquier tabla SQL existente `power_plant` usando el comando SQL:` DROP TABLE IF EXISTS power_plant` (tambien debemos que eliminar todos los ficheros asociados a la tabla, lo que podemos hacer con una operacion de sistema de archivos Databricks).

Una vez ejecutado el paso anterior, podemos registrar nuestro DataFrame como una tabla de SQL usando [sqlContext.registerDataFrameAsTable()](https://spark.apache.org/docs/1.6.2/api/python/pyspark.sql.html#pyspark.sql.SQLContext.registerDataFrameAsTable).

### Ejercicio 3(a)

Ejecutar la siguiente celda con el codigo ya preparado.

In [29]:
sqlContext.sql("DROP TABLE IF EXISTS power_plant")
dbutils.fs.rm("dbfs:/user/hive/warehouse/power_plant", True)
sqlContext.registerDataFrameAsTable(powerPlantDF, "power_plant")

Ahora que nuestro DataFrame existe como una tabla SQL, podemos explorarlo utilizando comandos SQL.

Para ejecutar SQL en una celda, utilizamos el operador `%sql`. La celda siguiente es un ejemplo del uso de SQL para consultar las filas de la tabla de SQL.

**NOTE**: `%sql` es una sentencia que solo funciona en los notebooks de Databricksis. Este ejecuta `sqlContext.sql()` y pasa los resultados a la funcion `display()`. Estas dos sentencias son equivalentes:

`%sql SELECT * FROM power_plant`

`display(sqlContext.sql("SELECT * FROM power_plant"))`

### Ejercicio 3(b)

Ejecutar la siguiente celda con el codigo ya preparado.

In [31]:
%sql
-- We can use %sql to query the rows
SELECT * FROM power_plant

### Ejercicio 3(c)

Usa el comando de SQL `desc` para describir el esquema ejecutando la siguiente celda.

In [33]:
%sql
desc power_plant

**Definicion de Esquema**

Una vez mas, nuestro esquema es el siguiente:

- AT = Atmospheric Temperature in C
- V = Exhaust Vacuum Speed
- AP = Atmospheric Pressure
- RH = Relative Humidity
- PE = Power Output

PE es nuestra variable objetivo. Este es el valor que intentamos predecir usando las otras mediciones.

*Referencia [UCI Machine Learning Repository Combined Cycle Power Plant Data Set](https://archive.ics.uci.edu/ml/datasets/Combined+Cycle+Power+Plant)*

Ahora vamos a realizar un analisis estadistico basico de todas las columnas.

Podemos obtener el DataFrame asociado a una tabla SQL usando el metodo [sqlContext.table()](https://spark.apache.org/docs/1.6.2/api/python/pyspark.sql.html#pyspark.sql.DataFrameReader.table) pasando como argumento el nombre de la tabla SQL. Una vez hecho esto, es posible usar el metodo nativo de un DataFrame [describe()](https://spark.apache.org/docs/1.6.2/api/python/pyspark.sql.html#pyspark.sql.DataFrame.describe) sin argumentos para calcular algunos estadisticos basicos para cara una de las columnas, como por ejemplo contar, media, max, min o la desviacion estandar.

In [36]:
df = sqlContext.table("power_plant")
display(df.describe())

##Parte 4: Visualizar los datos


Para entender nuestros datos, intentamos buscar correlaciones entre las diferentes caracteristicas y sus correspondientes etiquetas. Esto puede ser importante cuando seleccionamos un modelo. Por ejemplo, si una etiqueta y sus caracteristicas se correlacionan de forma lineal, un modelo lineal de regresion lineal obtendra un buen rendimiento; por el contrario si la relacion es no lineal, modelos mas complejos, como arboles de decision pueden ser una mejor opcion. Podemos utilizar herramientas de visualizacion para observar cada uno de los posibles predictores en relacion con la etiqueta como un grafico de dispersion para ver la correlacion entre ellos.

### Ejercicio 4(a)

** Anade las siguientes figuras: **
Vamos a ver si hay una correlacion entre la temperatura y la potencia de salida. Podemos utilizar una consulta SQL para crear una nueva tabla que contenga solo el de temperatura (AT) y potencia (PE), y luego usar un grafico de dispersion con la temperatura en el eje X y la potencia en el eje Y para visualizar la relacion (si la hay) entre la temperatura y la energia.

Realiza los siguientes pasos:

- Ejecuta la siguiente celda
- Haz clic en el menu desplegable junto al icono de "Bar Chart" y selecciona "Scatter" para convertir la tabla en un grafico de dispersion
- Haz click en "Plot Options..."
- En la caja de valores, haz clic en "Temperature" y arrastralo antes de "Power"
- Aplicar los cambios haciendo clic en el boton "Apply"
- Aumentar el tamano del grafico haciendo clic y arrastrando el control del tamano

In [38]:
%sql
select AT as Temperature, PE as Power from power_plant

Parece que hay una gran correlacion entre temperatura y power output. Esta correlacion es esperable gracias a la segunda ley de la termodinamica [thermal efficiency](https://en.wikipedia.org/wiki/Thermal_efficiency). Ir mas alla en este analisis queda fuera del ambito de esta practica.

### Ejercicio 4(b)

Usa una sentencia SQL para crear un grafico de dispersion entre las variables Power (PE) y Exhaust Vacuum Speed (V).

In [41]:
%sql
-- TO DO: Replace <FILL_IN> with the appropriate SQL command.

Ahora vamos a repetir este ejercicio con el resto de variables y la etiqueta Power Output.

### Ejercicio 4(c)

Usa una sentencia SQL para crear un grafico de dispersion entre las variables Power (PE) y Pressure (AP).

In [43]:
%sql
-- TO DO: Replace <FILL_IN> with the appropriate SQL command.
<FILL_IN>

### Ejercicio 4(d)

Usa una sentencia SQL para crear un grafico de dispersion entre las variables Power (PE) y Humidity (RH).

In [45]:
%sql
-- TO DO: Replace <FILL_IN> with the appropriate SQL command.
<FILL_IN>

##Parte 5: Preparacion de los datos

El siguiente paso es preparar los datos para aplicar la regresion. Dado que todo el dataset es numerico y consistente, esta sera una tarea sencilla y directa.

El objetivo es utilizar el metodo de regresion para determinar una funcion que nos de la potencia de salida como una funcion de un conjunto de caracteristicas de prediccion. El primer paso en la construccion de nuestra regresion es convertir las caracteristicas de prediccion de nuestro DataFrame a un vector de caracteristicas utilizando el metodo [pyspark.ml.feature.VectorAssembler()](https://spark.apache.org/docs/1.6.2/api/python/pyspark.ml.html#pyspark.ml.feature.VectorAssembler).

El VectorAssembler es una transformacion que combina una lista dada de columnas en una unico vector. Esta transformacion es muy util cuando queremos combinar caracteristicas en crudo de los datos con otras generadas al aplicar diferentes funciones sobre los datos en un unico vector de caracteristicas. Para integrar en un unico vector toda esta informacion antes de ejecutar un algoritmo de aprendizaje automatico, el VectorAssembler toma una lista con los nombres de las columnas de entrada (lista de strings) y el nombre de la columna de salida (string).

### Ejercicio 5(a)

- leer la documentacion y los ejemplos de uso de [VectorAssembler](https://spark.apache.org/docs/1.6.2/ml-features.html#vectorassembler)
- Convertir la tabla SQL `power_plant` en un `dataset` llamado datasetDF
- Establecer las columnas de entrada del VectorAssember: `["AT", "V", "AP", "RH"]`
- Establecer la columnas de salida como `"features"`

In [47]:
# TODO: Replace <FILL_IN> with the appropriate code
from pyspark.ml.feature import VectorAssembler

datasetDF = <FILL_IN>

vectorizer = VectorAssembler()
vectorizer.setInputCols(<FILL_IN>)
vectorizer.setOutputCol(<FILL_IN>)

In [48]:
# TEST
Test.assertEquals(set(vectorizer.getInputCols()), {"AT", "V", "AP", "RH"}, "Incorrect vectorizer input columns")
Test.assertEquals(vectorizer.getOutputCol(), "features", "Incorrect vectorizer output column")

##Parte 6: Modelar los datos

Ahora vamos a modelar nuestros datos para predecir que potencia de salida se dara cuando tenemos una serie de lecturas de los sensores

Nuestro primer modelo se basara en una simple regresion lineal ya que vimos algunos patrones lineales en nuestros datos en los graficos de dispersion durante la etapa de exploracion.

Necesitamos una forma de evaluar como de bien nuestro modelo de regresion lineal predice la produccion de potencia en funcion de parametros de entrada. Podemos hacer esto mediante la division de nuestros datos iniciales establecidos en un _Training set_ utilizado para entrenar a nuestro modelo y un _Test set_ utilizado para evaluar el rendimiento de nuestro modelo. Podemos usar el metodo nativo de los DataFrames [randomSplit()](https://spark.apache.org/docs/1.6.2/api/python/pyspark.sql.html#pyspark.sql.DataFrame.randomSplit) para dividir nuestro dataset. El metodo toma una lista de pesos y una semilla aleatoria opcional. La semilla se utiliza para inicializar el generador de numeros aleatorios utilizado por la funcion de division.

### Ejercicio 6(a)

Utiliza el metodo [randomSplit()](https://spark.apache.org/docs/1.6.2/api/python/pyspark.sql.html#pyspark.sql.DataFrame.randomSplit) para dividir `datasetDF` en trainingSetDF (80% del DataFrame de entrada) y testSetDF (20% del DataFrame de entrada), para poder reproducir siempre el mismo resultado, usar la semilla 1800009193L. Finalmente, cachea (cache()) cada datafrane en memoria para maximizar el rendimiento.

In [50]:
# TODO: Replace <FILL_IN> with the appropriate code.
# We'll hold out 20% of our data for testing and leave 80% for training
seed = 1800009193L
(split20DF, split80DF) = datasetDF.<FILL_IN>

# Let's cache these datasets for performance
testSetDF = <FILL_IN>
trainingSetDF = <FILL_IN>

In [51]:
# TEST
Test.assertEquals(trainingSetDF.count(), 38243, "Incorrect size for training data set")
Test.assertEquals(testSetDF.count(), 9597, "Incorrect size for test data set")

A continuacion vamos a crear un modelo de regresion lineal y utilizar su ayda para entender como entrenarlo. Ver la API de [Linear Regression](https://spark.apache.org/docs/1.6.2/api/python/pyspark.ml.html#pyspark.ml.regression.LinearRegression) para mas detalles.

### Ejercicio 6(b)

- Lee la documentacion y los ejemplos de [Linear Regression](https://spark.apache.org/docs/1.6.2/ml-classification-regression.html#linear-regression)
- Ejecuta la siguiente celda

In [53]:
# ***** LINEAR REGRESSION MODEL ****

from pyspark.ml.regression import LinearRegression
from pyspark.ml.regression import LinearRegressionModel
from pyspark.ml import Pipeline

# Let's initialize our linear regression learner
lr = LinearRegression()

# We use explain params to dump the parameters we can use
print(lr.explainParams())

La siguiente celda esta basada en [Spark ML Pipeline API for Linear Regression](https://spark.apache.org/docs/1.6.2/api/python/pyspark.ml.html#pyspark.ml.regression.LinearRegression).

El primer paso es establecer los valores de los parametros:
- Define el nombre de la columna a donde guardaremos la prediccion como "Predicted_PE"
- Define el nombre de la columna que contiene la etiqueta como "PE"
- Define el numero maximo de iteraciones a 100
- Define el parametro de regularizacion a 0.1

Ahora, crearemos el [ML Pipeline](https://spark.apache.org/docs/1.6.2/api/python/pyspark.ml.html#pyspark.ml.Pipeline) (flujo de ejecucion) y estableceremos las fases del pipeline como vectorizar y posteriormente aplicar el regresor lineal que hemos definido.

Finalmente, crearemos el modelo entrenandolo con el DataFrame `trainingSetDF`.

### Ejercicio 6(c)

- Lee la documentacion [Linear Regression](https://spark.apache.org/docs/1.6.2/api/python/pyspark.ml.html#pyspark.ml.regression.LinearRegression) documentation
- Completa y ejecuta la siguiente celda, asegurate de entender que es lo que sucede.

In [55]:
## TODO: Replace <FILL_IN> with the appropriate code
# Now we set the parameters for the method
lr.setPredictionCol(<FILL IN>)\
  .setLabelCol(<FILL IN>)\
  .setMaxIter(<FILL IN>)\
  .setRegParam(<FILL IN>)


# We will use the new spark.ml pipeline API. If you have worked with scikit-learn this will be very familiar.
lrPipeline = Pipeline()

lrPipeline.setStages([vectorizer, lr])

# Let's first train on the entire dataset to see what we get
lrModel = lrPipeline.fit(trainingSetDF)

Del articulo de Wikipedia [Linear Regression](https://en.wikipedia.org/wiki/Linear_regression) podemos leer:
> In statistics, linear regression is an approach for modeling the relationship between a scalar dependent variable \\( y \\) and one or more explanatory variables (or independent variables) denoted \\(X\\). In linear regression, the relationships are modeled using linear predictor functions whose unknown model parameters are estimated from the data. Such models are called linear models.

Los modelos de regresion lineal tienen muchos usos practicos. La mayoria de los cuales se clasifican en de las siguientes dos categorias:
- Si el objetivo es la prediccion o la reduccion de errores, la regresion lineal puede utilizarse para adaptar un modelo predictivo a un conjunto de datos observados \\(y\\) y \\(X\\). Despues de desarrollar un modelo de este tipo, dado un cierto valor  \\( X\\) del que no conocemos su valor de \\(y \\), el modelo ajustado se puede utilizarse para hacer una prediccion del valor del posible valor \\(y \\).
- Dada una variable \\(y\\) y un numero de variables \\( X_1 \\), ..., \\( X_p \\) que pueden estar relacionadas con \\(y\\), un analisis de regresion lineal puede ser aplicado a cuantificar como de fuerte es la relacion entre \\(y\\) y cada \\( X_j\\), para evaluar que \\( X_j \\) puede no tener ninguna relacion con \\(y\\), y de esta forma identificar que subconjuntos de \\( X_j \\) contienen informacion redundante sobre \\(y\\).

Como estamos interesados en ambos usos, nos gustaria para predecir la potencia de salida en funcion de las variables de entrada, y nos gustaria saber cuales de las variables de entrada estan debilmente o fuertemente correlacionadas con la potencia de salida.

Ya que una regresion lineal tan solo calcula la linea que minimiza el error cuadratico medio en el dataset de entrenamiento, dadas multiples dimensiones de entrada podemos expresar cada predictor como una funcion lineal en la forma:

\\[ y = a + b x_1 + b x_2 + b x_i ... \\]

donde \\(a\\) es el intercept (valor para el punto 0) y las \\(b\\) son los coeficientes.

Para expresar los coeficientes de esa linea podemos recuperar la etapa del Estimador del Modelo del pipeline y de expresar los pesos y el intercept de la funcion.

### Ejercicio 6(d)

Ejecuta la celda siguiente y asegurate que entiendes lo que sucede.

In [57]:
# The intercept is as follows:
intercept = lrModel.stages[1].intercept

# The coefficents (i.e., weights) are as follows:
weights = lrModel.stages[1].coefficients

# Create a list of the column names (without PE)
featuresNoLabel = [col for col in datasetDF.columns if col != "PE"]

# Merge the weights and labels
coefficents = zip(weights, featuresNoLabel)

# Now let's sort the coefficients from greatest absolute weight most to the least absolute weight
coefficents.sort(key=lambda tup: abs(tup[0]), reverse=True)

equation = "y = {intercept}".format(intercept=intercept)
variables = []
for x in coefficents:
    weight = abs(x[0])
    name = x[1]
    symbol = "+" if (x[0] > 0) else "-"
    equation += (" {} ({} * {})".format(symbol, weight, name))

# Finally here is our equation
print("Linear Regression Equation: " + equation)

### Ejercicio 6(e)

Ahora estudiaremos como se comportan nuestras predicciones en este modelo. Aplicamos nuestro modelo de regresion lineal para el 20% de los datos que hemos separado del conjunto de datos de entrada. La salida del modelo sera una columna de produccion de electricidad teorica llamada "Predicted_PE".

- Ejecuta la siguiente celda
- Desplazate por la tabla de resultados y observa como los valores de la columna de salida de corriente (PE) se comparan con los valores correspondientes en la salida de potencia predecida  (Predicted_PE)

In [59]:
# Apply our LR model to the test data and predict power output
predictionsAndLabelsDF = lrModel.transform(testSetDF).select("AT", "V", "AP", "RH", "PE", "Predicted_PE")

display(predictionsAndLabelsDF)

A partir de una inspeccion visual de las predicciones, podemos ver que estan cerca de los valores reales.

Sin embargo, nos gustaria disponer de una medida cientifica exacta de la bondad del modelo de regresion lineal. Para realizar esta medicion, podemos utilizar una metrica de evaluacion como la [Error cuadratico medio](https://en.wikipedia.org/wiki/Root-mean-square_deviation) (RMSE) para validar nuestro modelo lineal.

RSME se define como: \\( RMSE = \sqrt{\frac{\sum_{i = 1}^{n} (x_i - y_i)^2}{n}}\\) donde \\(y_i\\) es el valor observado \\(x_i\\) es el valor predecido

RMSE es una medida muy habitual para calcular las diferencias entre los valores predecidos por un modelo o un estimador y los valores realmente observados. Cuanto menor sea el RMSE, mejor sera nuestro modelo.

Spark ML Pipeline proporciona diferentes metricas para evaluar modelos de regresion, incluyendo [RegressionEvaluator()](https://spark.apache.org/docs/1.6.2/api/python/pyspark.ml.html#pyspark.ml.evaluation.RegressionEvaluator).

Despues de crerar una instancia de [RegressionEvaluator](https://spark.apache.org/docs/1.6.2/api/python/pyspark.ml.html#pyspark.ml.evaluation.RegressionEvaluator), fijaremos el nombre de la columna objetivo "PE" y  el nombre de la columna de prediccion a "Predicted_PE". A continuacion, invocaremos el evaluador en las predicciones.


### Ejercicio 6(f)

Ejecuta la celda siguiente y asegurate que entiendes lo que sucede.

In [61]:
# Now let's compute an evaluation metric for our test dataset
from pyspark.ml.evaluation import RegressionEvaluator

# Create an RMSE evaluator using the label and predicted columns
regEval = RegressionEvaluator(predictionCol="Predicted_PE", labelCol="PE", metricName="rmse")

# Run the evaluator on the DataFrame
rmse = regEval.evaluate(predictionsAndLabelsDF)

print("Root Mean Squared Error: %.2f" % rmse)

Otra medida de evaluacion estadistica muy util es el coeficiente de determinacion, que se denota \\(R ^ 2 \\) o \\(r ^ 2\\) y pronunciado "R cuadrado". Es un numero que indica la proporcion de la variacion en la variable dependiente que es predecible a partir de las variables independientes y proporciona una medida de lo bien que los resultados observados son replicados por el modelo, basado en la proporcion de la variacion total de los resultados explicada por el modelo. El coeficiente de determinacion va de 0 a 1 (mas cerca a 1), y cuanto mayor sea el valor, mejor es nuestro modelo.


Para calcular \\(r^2\\), hemos de ejecutar el evaluador `regEval.metricName: "r2"`

### Ejercicio 6(g)

Ejecuta la celda siguiente y asegurate que entiendes lo que sucede.

In [63]:
# Now let's compute another evaluation metric for our test dataset
r2 = regEval.evaluate(predictionsAndLabelsDF, {regEval.metricName: "r2"})

print("r2: {0:.2f}".format(r2))

En general, suponiendo una distribucion Gaussiana de errores, un buen modelo tendra 68% de las predicciones dentro de 1 RMSE y 95% dentro de 2 RMSE del valor real (ver http://statweb.stanford.edu/~susan/courses/s60/split/node60.html).

Vamos a examinar las predicciones y ver si un RMSE de 4,59 cumple este criterio.

Crearemos un nuevo DataFrame usando [selectExpr()](https://spark.apache.org/docs/1.6.2/api/python/pyspark.sql.html#pyspark.sql.DataFrame.selectExpr) para generar un conjunto de expresiones SQL, y registrar el DataFrame como una tabla de SQL utilizando [registerTempTable()](https://spark.apache.org/docs/1.6.2/api/python/pyspark.sql.html#pyspark.sql.DataFrame.registerTempTable).

### Ejercicio 6(h)

Ejecuta la celda siguiente y asegurate que entiendes lo que sucede.

In [65]:
# First we remove the table if it already exists
sqlContext.sql("DROP TABLE IF EXISTS Power_Plant_RMSE_Evaluation")
dbutils.fs.rm("dbfs:/user/hive/warehouse/Power_Plant_RMSE_Evaluation", True)

# Next we calculate the residual error and divide it by the RMSE
predictionsAndLabelsDF.selectExpr("PE", "Predicted_PE", "PE - Predicted_PE Residual_Error", "(PE - Predicted_PE) / {} Within_RSME".format(rmse)).registerTempTable("Power_Plant_RMSE_Evaluation")

Podemos utilizar sentencias SQL para explorar la tabla `Power_Plant_RMSE_Evaluation`. En primer lugar vamos a ver que datos en la tabla utilizando una sentencia SELECT de SQL.

### Exercise 6(i)

Ejecuta la celda siguiente y asegurate que entiendes lo que sucede.

In [67]:
%sql
SELECT * from Power_Plant_RMSE_Evaluation

Ahora podemos mostrar el RMSE como un histograma.

### Ejercicio 6(j)

Ejecuta los siguientes pasos:

- Ejecuta la siguiente celda
- Haz clic en el menu desplegable junto al icono "Bar chart" y selecciona "Histogram" para convertir la tabla en un histograma
- Haz clic en "Plot Options..."
- En la caja "All Fields:", haz clic "&lt;id&gt;" y arrastralo dentro de la caja "Keys:"
- Cambia el valor "Aggregation" a "COUNT"
- Aplicar los cambios haciendo clic en el boton Aplicar
- Aumentar el tamano del grafico haciendo clic y arrastrando el control del tamano

Observa que el histograma muestra claramente que el RMSE se centra alrededor de 0 con la gran mayoria de errores dentro de 2 RMSE.

In [69]:
%sql
-- Now we can display the RMSE as a Histogram
SELECT Within_RSME  from Power_Plant_RMSE_Evaluation

Usando una instruccion SELECT de SQL un poco mas compleja, podemos contar el numero de predicciones dentro de + o - 1,0 y + o - 2,0 y luego mostrar los resultados como un grafico circular.

### Ejercicio 6(k)

Ejecuta los siguientes pasos:  
  - Ejecutar la siguiente celda
  - Haz clic en el menu desplegable junto al icono de "Bar chart" y selecciona "Pie" para convertir la tabla en un grafico de sectores
  - Aumentar el tamano del grafico haciendo clic y arrastrando el control del tamano

In [71]:
%sql
SELECT case when Within_RSME <= 1.0 AND Within_RSME >= -1.0 then 1
            when  Within_RSME <= 2.0 AND Within_RSME >= -2.0 then 2 else 3
       end RSME_Multiple, COUNT(*) AS count
FROM Power_Plant_RMSE_Evaluation
GROUP BY case when Within_RSME <= 1.0 AND Within_RSME >= -1.0 then 1  when  Within_RSME <= 2.0 AND Within_RSME >= -2.0 then 2 else 3 end

### Conclusiones
A partir del pie chart, podemos ver que el 68% de nuestras predicciones de datos de prueba estan a 1 RMSE de los valores reales, y el 97% (68% + 29%) de nuestras predicciones de datos de prueba se encuentran a 2 RMSE. Por lo que el modelo es bastante decente.

##Parte 7: Ajustar y evaluar

Ahora que tenemos un primer modelo bastante bueno vamos a tratar de hacer uno aun mejor ajustando sus parametros. El proceso de ajustar un modelo se conoce como [Model Selection](https://spark.apache.org/docs/1.6.2/api/python/pyspark.ml.html#module-pyspark.ml.tuning) o [Hyperparameter Tuning](https://spark.apache.org/docs/1.6.2/api/python/pyspark.ml.html#module-pyspark.ml.tuning). Spark ML Pipeline hace que el proceso de ajuste sea sencillo.

Spark ML Pipeline soporta la seleccion de modelos usando herramientas herramientas como el [CrossValidator](https://spark.apache.org/docs/1.6.2/api/python/pyspark.ml.html#pyspark.ml.tuning.CrossValidator), que requiere los siguientes elementos:
- [Estimator](https://spark.apache.org/docs/1.6.2/api/python/pyspark.ml.html#pyspark.ml.Estimator): un algoritmo o un pipeline a ajustar
- [Conjunto de ParamMaps](https://spark.apache.org/docs/1.6.2/api/python/pyspark.ml.html#pyspark.ml.tuning.ParamGridBuilder): parametros para elegir, tambien conocido como _parameter grid_
- [Evaluator](https://spark.apache.org/docs/1.6.2/api/python/pyspark.ml.html#pyspark.ml.evaluation.Evaluator): metrica para medir que tan bien lo hace un modelo sobre los datos de entrenamiento

A un alto nivel, las herramientas de seleccion de modelos, tales como [CrossValidator](https://spark.apache.org/docs/1.6.2/api/python/pyspark.ml.html#pyspark.ml.tuning.CrossValidator) trabajan de la siguiente manera:

- Se separaran los datos de entrada en dos conjuntos entrenamiento y test.
- Para cada uno de estos pares (entrenamiento, test), hay iterar a traves del conjunto de ParamMaps:
    - Para cada [ParamMap](https://spark.apache.org/docs/1.6.2/api/python/pyspark.ml.html#pyspark.ml.tuning.ParamGridBuilder), se ajusta el [Estimador](https://spark.apache.org/docs/1.6.2/api/python/pyspark.ml.html#pyspark.ml.Estimator) usando dichos parametros, se obtiene el modelo ajustado, y se evaluar su rendimiento usando el [Evaluator] (https://spark.apache.org/docs/1.6.2/api/python/pyspark.ml.html#pyspark.ml.evaluation.Evaluator).
    - Seleccionan el mejor modelo producido por el conjunto de parametros.

El [Evaluator](https://spark.apache.org/docs/1.6.2/api/python/pyspark.ml.html#pyspark.ml.evaluation.Evaluator) puede ser por ejemplo un [RegressionEvaluator](https://spark.apache.org/docs/1.6.2/api/python/pyspark.ml.html#pyspark.ml.evaluation.RegressionEvaluator) para problemas de regresion. Como ayuda a construir el conjunto de parametros, los usuarios pueden utilizar la utilidad [ParamGridBuilder](https://spark.apache.org/docs/1.6.2/api/python/pyspark.ml.html#pyspark.ml.tuning.ParamGridBuilder).

Ten en cuenta que la validacion cruzada sobre una conjunto grande de parametros es costosa.

En el siguiente apartado llevaremos a cabo los siguientes pasos:
- Crear un [CrossValidator](https://spark.apache.org/docs/1.6.2/api/python/pyspark.ml.html#pyspark.ml.tuning.CrossValidator) utilizando un pipeline y un [RegressionEvaluator](https://spark.apache.org/docs/1.6.2/api/python/pyspark.ml.html#pyspark.ml.evaluation.RegressionEvaluator) que hemos creado anteriormente, y establecer el numero de pliegues (folds) a 5
- Crear una lista de 10 parametros de regularizacion
  - Crear una lista de 5 parametros de numero maximo de iteraciones
- Usar [ParamGridBuilder](https://spark.apache.org/docs/1.6.2/api/python/pyspark.ml.html#pyspark.ml.tuning.ParamGridBuilder) para construir un conjunto de parametros con los parametros de regularizacion y numero de iteraciones y anadir dicho conjunto al [CrossValidator](https://spark.apache.org/docs/1.6.2/api/python/pyspark.ml.html#pyspark.ml.tuning.CrossValidator)
- Ejecutar el [CrossValidator](https://spark.apache.org/docs/1.6.2/api/python/pyspark.ml.html#pyspark.ml.tuning.CrossValidator) para encontrar los parametros que producen el mejor modelo (es decir, mas bajo RMSE) y devolver el mejor modelo.

In [74]:
#TODO: Find the best parameter combination for a linear regression model
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

# We can reuse the RegressionEvaluator, regEval, to judge the model based on the best Root Mean Squared Error
# Let's create our CrossValidator with 3 fold cross validation
crossval = CrossValidator(estimator=lrPipeline, evaluator=regEval, numFolds=3)

# Let's tune over our regularization parameter from 0.01 to 0.10
regParam = [x / 100.0 for x in range(1, 11)]

# We'll create a paramter grid using the ParamGridBuilder, and add the grid to the CrossValidator
paramGrid = (ParamGridBuilder()
             .addGrid(lr.regParam, regParam)
             .build())
crossval.setEstimatorParamMaps(paramGrid)

# Now let's find and return the best model
cvModel = crossval.fit(trainingSetDF).bestModel

Ahora que ya hemos ajustado nuestro modelo de regression, vamos a ver cuales son los nuevos valores de RMSE y \\(r^2\\) que hemos obtenido en comparacion con los anteriores.

### Ejercicio 7(b)

Completa y ejecuta la siguiente celda

In [76]:
# TODO: Replace <FILL_IN> with the appropriate code.
# Now let's use cvModel to compute an evaluation metric for our test dataset: testSetDF
predictionsAndLabelsDF = <FILL_IN>

# Run the previously created RMSE evaluator, regEval, on the predictionsAndLabelsDF DataFrame
rmseNew = <FILL_IN>

# Now let's compute the r2 evaluation metric for our test dataset
r2New = <FILL_IN>

print("Original Root Mean Squared Error: {0:2.2f}".format(rmse))
print("New Root Mean Squared Error: {0:2.2f}".format(rmseNew))
print("Old r2: {0:2.2f}".format(r2))
print("New r2: {0:2.2f}".format(r2New))

In [77]:
# TEST
Test.assertEquals(round(rmse, 2), 4.59, "Incorrect value for rmse")
Test.assertEquals(round(rmseNew, 2), 4.59, "Incorrect value for rmseNew")
Test.assertEquals(round(r2, 2), 0.93, "Incorrect value for r2")
Test.assertEquals(round(r2New, 2), 0.93, "Incorrect value for r2New")